# Advanced RAG with Chains and Retrievers

In [4]:
# Loading the pdf document

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("The Power of Subconscious mind.pdf")

docs = loader.load()
docs

[Document(metadata={'source': 'The Power of Subconscious mind.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'The Power of Subconscious mind.pdf', 'page': 1}, page_content='Dr Joseph Murphy, a proponent of the New Thought movement, was born\non 20 May 1898, in a small town in the County of Cork, Ireland. He has\nauthored more than 30 books. His most famous work, The Power of Your\nSubconscious Mind, which was first published in 1963, was acclaimed as\none of the best self-help guides ever written. Dr Murphy died in December\n1981.\n1'),
 Document(metadata={'source': 'The Power of Subconscious mind.pdf', 'page': 2}, page_content='2'),
 Document(metadata={'source': 'The Power of Subconscious mind.pdf', 'page': 3}, page_content='Published by\nRupa Publications India Pvt. Ltd 2020\n7/16, Ansari Road, Daryaganj\nNew Delhi 110002\nEdition copyright © Rupa Publications India Pvt. Ltd 2020\nThe views and opinions expressed in this book are the author’s own and the facts are 

In [5]:
# Splitting into CHUNKS

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunk_documents = text_splitter.split_documents(docs)
chunk_documents


[Document(metadata={'source': 'The Power of Subconscious mind.pdf', 'page': 1}, page_content='Dr Joseph Murphy, a proponent of the New Thought movement, was born\non 20 May 1898, in a small town in the County of Cork, Ireland. He has\nauthored more than 30 books. His most famous work, The Power of Your\nSubconscious Mind, which was first published in 1963, was acclaimed as\none of the best self-help guides ever written. Dr Murphy died in December\n1981.\n1'),
 Document(metadata={'source': 'The Power of Subconscious mind.pdf', 'page': 2}, page_content='2'),
 Document(metadata={'source': 'The Power of Subconscious mind.pdf', 'page': 3}, page_content='Published by\nRupa Publications India Pvt. Ltd 2020\n7/16, Ansari Road, Daryaganj\nNew Delhi 110002\nEdition copyright © Rupa Publications India Pvt. Ltd 2020\nThe views and opinions expressed in this book are the author’s own and the facts are as reported by\nhim which have been verified to the extent possible, and the publishers are not in

In [6]:
# Vector Embedding

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name = "distilbert-base-nli-stsb-mean-tokens")

db = FAISS.from_documents(chunk_documents,embeddings)
db

### Designing ChatPrompt Template

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context.
Answer step by step in detail based on the provided input.
<context>
{context}
</context>
                                          
Question:{input}
                                        
""")

### LLM

In [8]:
import os
from constants import HUGGINGFACE_API_KEY
from langchain.llms import HuggingFaceHub
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACE_API_KEY

# Accessing the HuggingFace model (like distilGPT-2) from HuggingFace API
llm = HuggingFaceHub(repo_id="distilgpt2")


C:\Users\HP\AppData\Local\Temp\ipykernel_13624\2728335493.py:7: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(repo_id="distilgpt2")


### Chains



In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\nAnswer the following question based on the provided context.\nAnswer step by step in detail based on the provided input.\n<context>\n{context}\n</context>\n                                          \nQuestion:{input}\n                                        \n'), additional_kwargs={})])
| HuggingFaceHub(client=<InferenceClient(model='distilgpt2', timeout=None)>, repo_id='distilgpt2', task='text-generation')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

### Retriever

In [10]:
"""
Retriever: Fetches relevant documents or information based on a query.
There various types of Retrievers as well
"""


retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023384A0E570>, search_kwargs={})

### Retriever Chain

In [11]:
"""
Retriever Chain takes the user query, which is then passed to the Retriever to fetch the relevant document.
Those documents (and original Inputs) are then passed to an LLM to generate a Responsefrom langchain.chains import create_retrieval_chain
"""


from langchain.chains import create_retrieval_chain
retriever_chain = create_retrieval_chain(retriever,document_chain)

In [13]:
response = retriever_chain.invoke({"input":"Prayer is the formulation"})
response

{'input': 'Prayer is the formulation',
 'context': [Document(metadata={'source': 'The Power of Subconscious mind.pdf', 'page': 10}, page_content='wherein the individual seeks only communion with god? My work with\npeople has made it necessary for me to study the various approaches to\nprayer. I have experienced the power of prayer in my own life, and I have\ntalked and worked with many people who also have enjoyed the help of\nprayer. The problem usually is how to tell others how to pray. People who\nare in trouble have difficulty in thinking and acting reasonably. They need\nan easy formula to follow, an obviously workable pattern that is simple and\nspecific. Often, they must be led to approach the emergency.\nUnique feature of this book\nThe unique feature of this book is its down-to-earth practicality. Here you\nare presented with simple, usable techniques and formulas, which you can\neasily apply in your workaday world. I have taught these simple processes\nto men and women all ov

In [14]:
response["answer"]

'Human: \nAnswer the following question based on the provided context.\nAnswer step by step in detail based on the provided input.\n<context>\nwherein the individual seeks only communion with god? My work with\npeople has made it necessary for me to study the various approaches to\nprayer. I have experienced the power of prayer in my own life, and I have\ntalked and worked with many people who also have enjoyed the help of\nprayer. The problem usually is how to tell others how to pray. People who\nare in trouble have difficulty in thinking and acting reasonably. They need\nan easy formula to follow, an obviously workable pattern that is simple and\nspecific. Often, they must be led to approach the emergency.\nUnique feature of this book\nThe unique feature of this book is its down-to-earth practicality. Here you\nare presented with simple, usable techniques and formulas, which you can\neasily apply in your workaday world. I have taught these simple processes\nto men and women all over 